# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb9c82bc580>
├── 'a' --> tensor([[ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628]])
└── 'x' --> <FastTreeValue 0x7fb9c82bc040>
    └── 'c' --> tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                        [-0.6960, -2.1471, -0.2288, -0.1620],
                        [ 2.1034,  0.2395,  1.0267,  0.6992]])

In [4]:
t.a

tensor([[ 1.2853, -0.6322, -0.6709],
        [-0.6807, -0.1294, -0.2628]])

In [5]:
%timeit t.a

59.5 ns ± 0.0342 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb9c82bc580>
├── 'a' --> tensor([[ 0.2159,  0.4805,  0.9980],
│                   [-1.2481,  0.7542,  1.4866]])
└── 'x' --> <FastTreeValue 0x7fb9c82bc040>
    └── 'c' --> tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                        [-0.6960, -2.1471, -0.2288, -0.1620],
                        [ 2.1034,  0.2395,  1.0267,  0.6992]])

In [7]:
%timeit t.a = new_value

59 ns ± 0.0702 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628]]),
    x: Batch(
           c: tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                      [-0.6960, -2.1471, -0.2288, -0.1620],
                      [ 2.1034,  0.2395,  1.0267,  0.6992]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2853, -0.6322, -0.6709],
        [-0.6807, -0.1294, -0.2628]])

In [11]:
%timeit b.a

54.5 ns ± 0.024 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3147,  1.5186, -0.9530],
               [-2.0315, -0.9509,  0.6998]]),
    x: Batch(
           c: tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                      [-0.6960, -2.1471, -0.2288, -0.1620],
                      [ 2.1034,  0.2395,  1.0267,  0.6992]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.223 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 328 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb8edfe8c10>
├── 'a' --> tensor([[[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]],
│           
│                   [[ 1.2853, -0.6322, -0.6709],
│                    [-0.6807, -0.1294, -0.2628]]])
└── 'x' --> <FastTreeValue 0x7fb9c422d760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.3 µs ± 52.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb8edfe89a0>
├── 'a' --> tensor([[ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628],
│                   [ 1.2853, -0.6322, -0.6709],
│                   [-0.6807, -0.1294, -0.2628]])
└── 'x' --> <FastTreeValue 0x7fb8edfe8cd0>
    └── 'c' --> tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                        [-0.6960, -2.1471, -0.2288, -0.1620],
                 

In [23]:
%timeit t_cat(trees)

32 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

70.9 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]],
       
               [[ 1.2853, -0.6322, -0.6709],
                [-0.6807, -0.1294, -0.2628]]]),
    x: Batch(
           c: tensor([[[-0.0800,  0.3897, -0.3020, -2.0715],
                       [-0.6960, -2.1471, -0.2288, -0.1620],
                       [ 2.1034,  0.2395,  1.0267,  0.6992]],
         

In [26]:
%timeit Batch.stack(batches)

80.3 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628],
               [ 1.2853, -0.6322, -0.6709],
               [-0.6807, -0.1294, -0.2628]]),
    x: Batch(
           c: tensor([[-0.0800,  0.3897, -0.3020, -2.0715],
                      [-0.6960, -2.1471, -0.2288, -0.1620],
                      [ 2.1034,  0.2395,  1.0267,  0.6992],
                      [-0.0800,  0.3897, -0.3020, -2.0715],
                      [-0.6960, -

In [28]:
%timeit Batch.cat(batches)

147 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

332 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
